In [3]:
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import time
import os

# ======================= Fetch Crypto Data ======================= #
def fetch_crypto_data(symbol, interval, limit=100, retries=3, delay=2):
    url = f"https://api.binance.com/api/v3/klines?symbol={symbol}USDT&interval={interval}&limit={limit}"
    
    for attempt in range(retries):
        try:
            print(f"Fetching data for {symbol} at {interval} interval...")
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            
            df = pd.DataFrame(data, columns=[
                'timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                'quote_asset_volume', 'trades', 'taker_buy_base', 'taker_buy_quote', 'ignore'
            ])
            
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            df['close'] = df['close'].astype(float)
            print("✅ Data fetched successfully!")
            return df[['close']]
        
        except requests.exceptions.RequestException as e:
            print(f"⚠️ API Error (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(delay)
    
    print("❌ Failed to fetch data after multiple retries.")
    return None

# ======================= Prepare Data ======================= #
def prepare_data(df, time_steps=20):
    if len(df) <= time_steps:
        raise ValueError(f"❌ Dataset too small for time_steps={time_steps}. Needs at least {time_steps + 1} rows.")
    
    print(f"📊 Preparing data with {time_steps} time steps...")
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    
    X, y = [], []
    for i in range(len(df_scaled) - time_steps):
        X.append(df_scaled[i:i + time_steps])
        y.append(df_scaled[i + time_steps])
    
    X, y = np.array(X), np.array(y)
    print(f"✅ Data prepared! Shape: X={X.shape}, y={y.shape}")
    return X, y, scaler

# ======================= Build LSTM Model ======================= #
def build_lstm_model(input_shape):
    print("🔧 Building LSTM model...")
    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        LSTM(50, return_sequences=False, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    print("✅ Model built successfully!")
    return model

# ======================= Train & Save Model ======================= #
def train_and_save_model(symbol, interval, epochs=100, batch_size=16, time_steps=20, model_filename="crypto_model.h5"):
    print("🚀 Starting model training...")
    
    # Fetch Data
    df = fetch_crypto_data(symbol, interval)
    if df is None:
        print("❌ No data fetched. Exiting training.")
        return None
    
    # Prepare Data
    try:
        X, y, scaler = prepare_data(df, time_steps)
    except ValueError as e:
        print(e)
        return None
    
    # Split Data
    train_size = int(len(X) * 0.8)
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]
    
    # Build Model
    model = build_lstm_model((X.shape[1], X.shape[2]))
    
    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    model_checkpoint = ModelCheckpoint(
        model_filename, save_best_only=True, mode='min', verbose=1
    )

    # Train Model
    try:
        history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=epochs, batch_size=batch_size, verbose=1,
                            callbacks=[early_stopping, model_checkpoint])
        print("✅ Training completed. Proceeding to save the model...")

        # Load best model after training
        model = load_model(model_filename)
        model.save(model_filename)
        
        # Save Scaler
        scaler_filename = model_filename.replace(".h5", "_scaler.pkl")
        joblib.dump(scaler, scaler_filename)
        print(f"✅ Model saved as {model_filename}, Scaler saved as {scaler_filename}")

        return model_filename, scaler_filename
    
    except Exception as e:
        print(f"❌ Training failed with error: {e}")
        return None

# ======================= Evaluate Model ======================= #
def evaluate_model(model, X_test, y_test, scaler):
    print("📈 Evaluating model...")
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test_original = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test_original, predictions))
    mae = mean_absolute_error(y_test_original, predictions)
    
    print(f"✅ RMSE: {rmse:.4f}")
    print(f"✅ MAE: {mae:.4f}")

# ======================= Load & Predict ======================= #
def load_and_predict(model_filename, symbol, interval, time_steps=20):
    print(f"🔍 Loading model {model_filename} for prediction...")
    
    df = fetch_crypto_data(symbol, interval)
    if df is None:
        print("❌ No data fetched for prediction.")
        return None
    
    try:
        X, _, scaler = prepare_data(df, time_steps)
        model = load_model(model_filename)
        predictions = model.predict(X)
        predictions = scaler.inverse_transform(predictions)
        
        print("✅ Predictions completed!")
        return df.index[-len(predictions):], predictions
    
    except Exception as e:
        print(f"❌ Prediction failed with error: {e}")
        return None

# ======================= Run Training ======================= #
if __name__ == "__main__":
    # Set Parameters
    symbol = "BTC"
    interval = "1h"
    epochs = 10
    batch_size = 8
    time_steps = 20
    model_filename = "crypto_model.h5"

    # Train Model
    model_path, scaler_path = train_and_save_model(symbol, interval, epochs, batch_size, time_steps, model_filename)

    # Evaluate Model
    if model_path:
        print("🎯 Model training completed successfully!")
    else:
        print("❌ Model training failed.")


🚀 Starting model training...
Fetching data for BTC at 1h interval...
✅ Data fetched successfully!
📊 Preparing data with 20 time steps...
✅ Data prepared! Shape: X=(80, 20, 1), y=(80, 1)
🔧 Building LSTM model...


c:\Users\yasir\Desktop\Python Projects\LSTM\lenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ Model built successfully!
Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.0620
Epoch 1: val_loss improved from inf to 0.74594, saving model to crypto_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 113ms/step - loss: 1.0506 - val_loss: 0.7459
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.7543
Epoch 2: val_loss improved from 0.74594 to 0.66439, saving model to crypto_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.7542 - val_loss: 0.6644
Epoch 3/10
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6564
Epoch 3: val_loss improved from 0.66439 to 0.55602, saving model to crypto_model.h5


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.3657 - val_loss: 0.3281
Epoch 8/10
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.3214
Epoch 8: val_loss improved from 0.32806 to 0.28617, saving model to crypto_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.3216 - val_loss: 0.2862
Epoch 9/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2910
Epoch 9: val_loss improved from 0.28617 to 0.24961, saving model to crypto_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.2904 - val_loss: 0.2496
Epoch 10/10
5/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.2754
Epoch 10: val_loss improved from 0.24961 to 0.22907, saving model to crypto_model.h5


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.2648 - val_loss: 0.2291


✅ Training completed. Proceeding to save the model...
✅ Model saved as crypto_model.h5, Scaler saved as crypto_model_scaler.pkl
🎯 Model training completed successfully!
